In [2]:
#### Setting up Spark for colab space, code provided by Professor Othman

import os
#Install Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Set JAVA_HOME path variable in Linux
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version


#Install Spark
#download Spark file
!wget -q http://apache.osuosl.org/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
#extract the file
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
#Set SPARK-HOME path variable in Linux
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"


#install findspark package
!pip install -q findspark

update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode
openjdk version "1.8.0_252"
OpenJDK Runtime Environment (build 1.8.0_252-8u252-b09-1~18.04-b09)
OpenJDK 64-Bit Server VM (build 25.252-b09, mixed mode)


In [3]:
import findspark
findspark.init()

In [4]:
# create entry points to spark
try:
    sc.stop()
except:
    pass
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext
conf = SparkConf().setAppName("lecture10").setMaster("local[*]")
sc=SparkContext(conf = conf)
spark = SparkSession(sparkContext=sc)
sqlContext = SQLContext(sc)

In [5]:
spark

In [76]:
# hardcoded variables telling our program which states, years, and tables are needed from NIBRS dataa

state_list = ["alabama", "arizona"]
state_abbr = ["AL", "AZ"]
csv = ["nibrs_offense", "nibrs_arrestee"]
firstyear = 1991
lastyear = 2018

In [79]:
import pandas as pd

df_all = []   # empty list of all read in tables
url_base="https://raw.githubusercontent.com/roched1atwit/CS3800_4050/master/data/"

for j in range(0, len(csv)):
    df_list = []
    for state_i in range(0,len(state_list)):
        for year in range(firstyear,lastyear+1):
            url = url_base+state_list[state_i]+'/'+state_abbr[state_i]+'-'+str(year)
            url_c = url + '/' + csv[j] + '.csv'
            print("Reading " + url_c)
            #print(url_c)
            try:
                temp_df = pd.read_csv(url_c, error_bad_lines=False)
            except:
                url_c = url + '/' + csv[j].upper() + '.csv'
                try:
                    temp_df = pd.read_csv(url_c, error_bad_lines=False)
                    temp_df.columns = map(str.lower, temp_df.columns)
                except:
                    print(url_c + " did not exist")
            
            temp_df['state'] = state_abbr[state_i]
            df_list.append(temp_df)
            #print(df_list)
            #break
    df_all.append(pd.concat(df_list))
        
#AL2018_OFFENSE=pd.read_csv(url,  error_bad_lines=False)
#AL2018_OFFENSE

Reading https://raw.githubusercontent.com/roched1atwit/CS3800_4050/master/data/alabama/AL-1991/nibrs_offense.csv
Reading https://raw.githubusercontent.com/roched1atwit/CS3800_4050/master/data/alabama/AL-1992/nibrs_offense.csv
Reading https://raw.githubusercontent.com/roched1atwit/CS3800_4050/master/data/alabama/AL-1993/nibrs_offense.csv
https://raw.githubusercontent.com/roched1atwit/CS3800_4050/master/data/alabama/AL-1993/NIBRS_OFFENSE.csv did not exist
Reading https://raw.githubusercontent.com/roched1atwit/CS3800_4050/master/data/alabama/AL-1994/nibrs_offense.csv
https://raw.githubusercontent.com/roched1atwit/CS3800_4050/master/data/alabama/AL-1994/NIBRS_OFFENSE.csv did not exist
Reading https://raw.githubusercontent.com/roched1atwit/CS3800_4050/master/data/alabama/AL-1995/nibrs_offense.csv
https://raw.githubusercontent.com/roched1atwit/CS3800_4050/master/data/alabama/AL-1995/NIBRS_OFFENSE.csv did not exist
Reading https://raw.githubusercontent.com/roched1atwit/CS3800_4050/master/data

In [80]:
df_all

[       offense_id  incident_id  ...  state data_year
 0          133326       130114  ...     AL       NaN
 1          133190       129982  ...     AL       NaN
 2          133130       129925  ...     AL       NaN
 3          133129       129924  ...     AL       NaN
 4          133128       129923  ...     AL       NaN
 ...           ...          ...  ...    ...       ...
 22795   123983060    100512293  ...     AZ    2018.0
 22796   123986945    100512306  ...     AZ    2018.0
 22797   123983074    100512311  ...     AZ    2018.0
 22798   123979075    100512324  ...     AZ    2018.0
 22799   123983102    100512338  ...     AZ    2018.0
 
 [4126100 rows x 10 columns],
       arrestee_id  incident_id  ...  state data_year
 0            1614       129882  ...     AL       NaN
 1            1615       130130  ...     AL       NaN
 2            1650       130935  ...     AL       NaN
 3            1649       130924  ...     AL       NaN
 4            1648       130912  ...     AL       

In [81]:
df_all[0]

,offense_id,incident_id,offense_type_id,attempt_complete_flag,location_id,num_premises_entered,method_entry_code,ff_line_number,state,data_year
0,133326,130114,36,C,20,NaN,NaN,NaN,AL,NaN
1,133190,129982,49,C,20,NaN,F,NaN,AL,NaN
2,133130,129925,49,C,20,NaN,F,NaN,AL,NaN
3,133129,129924,40,C,23,NaN,NaN,NaN,AL,NaN
4,133128,129923,45,C,20,NaN,NaN,NaN,AL,NaN
...,...,...,...,...,...,...,...,...,...,...
22795,123983060,100512293,23,C,24,NaN,NaN,NaN,AZ,2018.0
22796,123986945,100512306,5,C,13,NaN,NaN,NaN,AZ,2018.0
22797,123983074,100512311,63,C,20,NaN,NaN,NaN,AZ,2018.0
22798,123979075,100512324,21,C,18,NaN,NaN,NaN,AZ,2018.0


In [84]:
from pyspark.sql.types import *

incident_df = StructType([ StructField("offense_id", IntegerType(), True)\

                       ,StructField("incident_id", IntegerType(), True)\

                       ,StructField("offense_type_id", IntegerType(), True)\

                       ,StructField("attempt_complete_flag", StringType(), True)\

                       ,StructField("location_id", IntegerType(), True)\

                       ,StructField("num_premises_entered", DoubleType(), True)\

                       ,StructField("method_entry_code", StringType(), True)\

                       ,StructField("ff_line_number", DoubleType(), True)\

                       ,StructField("state", StringType(), True)\

                       ,StructField("data_year", DoubleType(), True)])

In [85]:
df = spark.createDataFrame(df_all[0],schema=mySchema)